# Unboundedness detection often fails for the exponential cone

Silvapulle (1981) gave simple criteria under which the MLE does not exist for the logistic regression model.
In this context it is said that the MLE exists, if it is finite and unique. Since in the 
example given by Silavpulle (1981) the data is separated, the problem is unbounded. 
The example is chosen to demonstrate that the unboundedness detection for the exponential cone often fails.

- **y** a factor with the levels `case` and `none-case`, giving the outcome of a standardized psychiatric interview.
- **ghqs** an integer giving the general health questionnaire score.

In [1]:
import cvxpy as cp
import cvxpy.settings as s
import numpy as np
import pprint
pp = pprint.PrettyPrinter(depth=4)

## Generate separated data

In [2]:
X = np.matrix("1 2 ; 1 4 ; 1 5 ; 1 5 ; 1 5 ; 1 7 ; 1 7 ; 1 10 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 0 ; 1 1 ; 1 1 ; 1 1 ; 1 1 ; 1 1 ; 1 1 ; 1 1 ; 1 1 ; 1 2")
Y = np.array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[X.shape, Y.shape]

[(35, 2), (35,)]

## Logistic regression

In [3]:
def loglike(X, y, beta):
    eta = X.dot(beta)
    return eta.dot(y).item() - np.sum(np.log(1 + np.exp(eta)))

In [4]:
beta = cp.Variable(X.shape[1])
log_likelihood = cp.sum(cp.multiply(Y, X @ beta) - cp.logistic(X @ beta))
problem = cp.Problem(cp.Maximize(log_likelihood))

### ECOS

In [5]:
problem.solve(solver = "ECOS", reltol = 1e-32)
beta_ecos = beta.value
print(f"Problem status: {problem.status}")
print("\nInfo:")
pp.pprint(problem.solution.attr["solver_specific_stats"]["info"])
print("\nCoefficients:")
print(beta_ecos)

Problem status: optimal

Info:
{'dcost': 1.3862953226819956,
 'dinf': 0.0,
 'dinfres': nan,
 'dres': 4.452163947615004e-10,
 'exitFlag': 0,
 'gap': 4.3774859605216765e-09,
 'infostring': 'Optimal solution found',
 'iter': 32,
 'mi_iter': -1,
 'numerr': 0,
 'pcost': 1.386295336242354,
 'pinf': 0.0,
 'pinfres': 0.8209648886264107,
 'pres': 5.798838790207396e-10,
 'r0': 1e-08,
 'relgap': 3.157686453166974e-09,
 'timing': {'runtime': 0.0018657880000000002,
            'tsetup': 0.000101291,
            'tsolve': 0.001764497}}

Coefficients:
[-38.28689184  19.14344586]


### SCS

In [6]:
problem.solve(solver = "SCS", max_iters = 200000)
beta_scs = beta.value
print(f"Problem status: {problem.status}")
print("\nInfo:")
pp.pprint(problem.solution.attr["solver_specific_stats"]["info"])
print("\nCoefficients:")
print(beta_scs)

Problem status: optimal

Info:
{'accel_time': 17.3171960000021,
 'accepted_accel_steps': 2,
 'comp_slack': 3.3787373813011824e-07,
 'cone_time': 27752.63602400004,
 'dobj': 1.387025136547876,
 'gap': 9.988237454417664e-06,
 'iter': 122450,
 'lin_sys_time': 425.797392000008,
 'pobj': 1.3870151483104216,
 'rejected_accel_steps': 1112,
 'res_dual': 2.1863802012257032e-05,
 'res_infeas': 32.443551798780796,
 'res_pri': 0.0006354718076612859,
 'res_unbdd_a': nan,
 'res_unbdd_p': nan,
 'scale': 0.1,
 'scale_updates': 0,
 'setup_time': 0.228409,
 'solve_time': 28349.882642,
 'status': 'solved',
 'status_val': 1}

Coefficients:
[-23.73555382  11.86772503]


### MOSEK

In [7]:
problem.solve(solver = "MOSEK")
beta_mosek = beta.value
print(f"Problem status: {problem.status}")
print("\nInfo:")
pp.pprint(problem.solution.attr)
print("\nCoefficients:")
print(beta_mosek)

Problem status: optimal

Info:
{'solve_time': 0.012694120407104492}

Coefficients:
[-40.10371589  20.05185876]


## Compare results

In [8]:
logli_ecos = loglike(X, Y, beta_mosek)
logli_scs = loglike(X, Y, beta_scs)
logli_mosek = loglike(X, Y, beta_mosek)
# Just to make it more obvious that [-2 * Inf, Inf] is the solution.
some_large_number = 60  # not that large to avoid overflow
guess_beta = [-2 * some_large_number, some_large_number]
logli_better_beta = loglike(X, Y, guess_beta)

In [9]:
np.argmax([logli_ecos, logli_scs, logli_mosek, logli_better_beta])

3

In [10]:
cp.__version__

'1.2.0'